In [9]:
using Query
using DataFrames
using CSVFiles

In [8]:
tests = split("dist ord extrapo")
tasks = split("dist point")

test_tasks = [joinpath("results", task*"_"*test*".csv") for task in tasks for test in tests]

6-element Array{String,1}:
 "results/dist_dist.csv"    
 "results/dist_ord.csv"     
 "results/dist_extrapo.csv" 
 "results/point_dist.csv"   
 "results/point_ord.csv"    
 "results/point_extrapo.csv"

In [10]:
filename =  "results/dist_extrapo.csv" 
df = DataFrame(load(filename))


,method,perp,perp_hue,perp_sat,perp_val,mse_to_distmode,mse_to_distmean
1,Direct Non-extrapolating,2.10279e9,156.552,3801.75,3533.08,0.117389,0.0636989
2,Direct-smoothed Non-extrapolating,8.70003e5,48.7585,157.652,113.18,0.0891889,0.0636972
3,SOWE Extrapolating,9.7992e5,51.3755,163.695,116.519,0.138944,0.0793886
4,SOWE Non-extrapolating,8.62954e5,48.4645,156.717,113.618,0.112753,0.0753511
5,SOWE-smoothed Extrapolating,9.50381e5,50.7427,161.952,115.648,0.0975256,0.078681
6,SOWE-smoothed Non-extrapolating,8.55923e5,48.4129,156.375,113.059,0.0911059,0.0751902
7,CNN Extrapolating,1.0345e6,52.4537,163.146,120.886,0.141667,0.0812984
8,CNN Non-extrapolating,9.03985e5,49.9958,157.114,115.083,0.150718,0.0755267
9,CNN-smoothed Extrapolating,1.07021e6,53.0244,164.243,122.887,0.115777,0.0820637
10,CNN-smoothed Non-extrapolating,9.31776e5,50.9268,157.887,115.883,0.0970555,0.0759993


In [46]:
method_name(method)::String = first(split(method))
extrapo_res = @from ii in df begin
    @where contains(ii.method, "Extrapolating")
    @select {method=method_name(ii.method), ii.perp}
    @collect DataFrame
end
nonextrapo_res = @from ii in df begin
    @where contains(ii.method, "Non-extrapolating")
    @select {method=method_name(ii.method), ii.perp}
    @collect DataFrame
end
@show extrapo_res 
@show nonextrapo_res;

extrapo_res = 6×2 DataFrames.DataFrame
│ Row │ method        │ perp      │
├─────┼───────────────┼───────────┤
│ 1   │ SOWE          │ 9.7992e5  │
│ 2   │ SOWE-smoothed │ 9.50381e5 │
│ 3   │ CNN           │ 1.0345e6  │
│ 4   │ CNN-smoothed  │ 1.07021e6 │
│ 5   │ RNN           │ 3.42562e6 │
│ 6   │ RNN-smoothed  │ 3.39419e6 │
nonextrapo_res = 8×2 DataFrames.DataFrame
│ Row │ method          │ perp      │
├─────┼─────────────────┼───────────┤
│ 1   │ Direct          │ 2.10279e9 │
│ 2   │ Direct-smoothed │ 8.70003e5 │
│ 3   │ SOWE            │ 8.62954e5 │
│ 4   │ SOWE-smoothed   │ 8.55923e5 │
│ 5   │ CNN             │ 9.03985e5 │
│ 6   │ CNN-smoothed    │ 9.31776e5 │
│ 7   │ RNN             │ 2.02091e6 │
│ 8   │ RNN-smoothed    │ 1.99849e6 │


In [54]:
@from nonext in nonextrapo_res begin
    @join ext in extrapo_res on nonetx.method equals ext.method
    @select {nonext.method, nonextrapolating_perp=nonext.perp, extrapolating_perp = ext.perp}
    @collect DataFrame
end

LoadError: [91mThe keys in the join clause have different types, Any and String.[39m

In [52]:
nonextrapo_res.columns

2-element Array{Any,1}:
 String["Direct", "Direct-smoothed", "SOWE", "SOWE-smoothed", "CNN", "CNN-smoothed", "RNN", "RNN-smoothed"]
 [2.10279e9, 8.70003e5, 8.62954e5, 8.55923e5, 9.03985e5, 9.31776e5, 2.02091e6, 1.99849e6]                  

In [53]:
extrapo_res.columns

2-element Array{Any,1}:
 String["SOWE", "SOWE-smoothed", "CNN", "CNN-smoothed", "RNN", "RNN-smoothed"]
 [9.7992e5, 9.50381e5, 1.0345e6, 1.07021e6, 3.42562e6, 3.39419e6]             